In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from Trainer import Trainer

from WaveValueDataset import WaveValueDataset

from torch.utils.data.dataloader import DataLoader

from Trainer import SequenceTrainer
from WaveModel import PredictionModel
# block_size = 512

mydataset = WaveValueDataset('full',161)
# M,N,n_embd,n_hidden,n_head,n_layer,dropout=0.2
model = PredictionModel(mic_num=6,tot_timesteps=160,n_embd=80,n_hidden=120,n_head=8,n_layer=4,dropout=0.2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
m=model.to(device)
train=SequenceTrainer(model,mydataset,eval_iters=5,batch_size=2,device=device)
train.to('cpu')

print('device : ', device)

Initialized dataset with 10 examples
device :  cpu


In [7]:
train.model.train()
train.optimization(epochs=10,eval_interval=2,lr=1e-3,loadingbar=False)

Gonna return : torch.Size([3, 6, 161])
Gonna return : torch.Size([3, 6, 161])
Idx shape : torch.Size([2, 3, 6, 160])
Idx shape : torch.Size([2, 160, 80])


ValueError: not enough values to unpack (expected 4, got 3)